In [1]:
# imports
import requests
# import sqlalchemy as db
# import pandas as pd
# from bs4 import BeautifulSoup as bs
from alchemyDB import *
from create_objects import *
import sqlalchemy
import time
import re
import pprint
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.expression import func
import pickle

In [3]:
## Run this to recreate the DB
# make_db()

In [2]:
engine = create_engine('sqlite:///boardgames.db')

Session = sessionmaker(bind=engine, autoflush=False)
session = Session()

In [3]:
max_id = session.query(func.max(Game.id)).first()[0]

In [4]:
max_id

1919

In [5]:
expansions=pickle.load(open('expansions.p','rb'))

In [6]:
check=pickle.load(open('check.p','rb'))

In [7]:
notgames=pickle.load(open('notgames.p','rb'))

In [42]:
print(expansions)
print(check)
print(notgames)

[122220, 5814, 5815, 164592, 43490, 125864, 184213, 186885, 282531, 206374, 65712, 165833, 177, 16109, 11568, 79843, 2058, 26916, 583, 597, 3146, 370, 5754, 27759, 5359, 22909, 21850, 10170, 12838, 22771, 6441, 9536, 19362, 22870, 17151, 10169, 10473, 3618, 22952, 22929, 22927, 32619, 13119, 27727, 36753, 269482, 63081, 186885, 184215, 85985, 2949, 2950, 264297, 197247, 2005, 12391, 85985, 2949, 2950, 264297, 197247, 2005, 12391, 3283, 2622, 8113, 15225, 60290, 1526, 25946, 25948, 38677, 59417, 207090, 261065, 282517, 266190, 266189, 124304, 61570, 61571, 104963, 30889, 71082, 68119, 124304, 178037, 231140, 5872, 111363, 124456, 147313, 233083, 191960, 195997, 187247, 227, 155192, 1677, 172877, 262716, 144576, 67707, 147119, 215430, 253107, 219234, 186885, 184215, 64895, 1685, 243258, 2575, 3043, 26105, 69988, 109580, 111775, 154473, 56839, 56841, 156604, 56840, 59644, 59645, 173574, 42040, 94856, 173574, 123137, 4347, 20030, 228784, 163187, 1020, 167903, 168260, 3260, 159349, 159351, 

In [17]:
i

415

In [38]:
import csv

game_ids=[]
with open('2019-07-08.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            pass
        else:
            game_ids.append(int(row[0]))
        line_count+=1
    print(f'Processed {line_count} lines.')

Processed 17314 lines.


In [30]:
game_ids

[174430,
 161936,
 182028,
 167791,
 12333,
 187645,
 220308,
 169786,
 233078,
 193738,
 120677,
 115746,
 224517,
 84876,
 162886,
 173346,
 180263,
 124361,
 183394,
 205637,
 3076,
 28720,
 102794,
 96848,
 31260,
 164928,
 205059,
 177736,
 175914,
 170216,
 221107,
 209010,
 55690,
 2651,
 25613,
 164153,
 126163,
 230802,
 72125,
 35677,
 121921,
 171623,
 266192,
 216132,
 124742,
 68448,
 237182,
 185343,
 122515,
 28143,
 18602,
 62219,
 159675,
 110327,
 157354,
 205896,
 12493,
 93,
 146021,
 125153,
 201808,
 178900,
 40834,
 163412,
 172386,
 37111,
 132531,
 73439,
 2511,
 144733,
 42,
 146508,
 102680,
 30549,
 36218,
 161533,
 103885,
 521,
 150376,
 127023,
 198928,
 200680,
 191189,
 170042,
 175155,
 104162,
 34635,
 229853,
 236457,
 196340,
 161970,
 147020,
 172287,
 9609,
 146652,
 155426,
 148949,
 103343,
 182874,
 128882,
 194655,
 14996,
 43111,
 17133,
 155821,
 70149,
 14105,
 123260,
 203993,
 155068,
 126042,
 77423,
 221194,
 224037,
 171131,
 192135,
 

In [ ]:
for i in game_ids:
    if (i not in expansions) & (i not in current_ids):
        address = f'https://boardgamegeek.com/xmlapi2/thing?id={i}&type=boardgame&stats=1'
        req = requests.get(address)
        if re.search(b'id=',req.content):
            expansions.extend(instantiate_games(req, i))
        elif re.search(b'Rate limit exceeded.', req.content):
            print('Zzzz')
            time.sleep(10)
            i-=1
        else:
            notgames.append(i)

    if i%100==0:
        print(f'100 more games! {get_game_collection()} collected')
        try:
            pickle.dump(expansions, open( "expansions.p", "wb+" ) )
            pickle.dump(check, open("check.p","wb+"))
            pickle.dump(notgames, open("notgames.p","wb+"))
        except:
            print('No pickle for you')

100 more games! 1567 collected
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
100 more games! 1647 collected
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
100 more games! 1821 collected
100 more games! 1825 collected
Zzzz
Zzzz
100 more games! 1833 collected
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
100 more games! 1906 collected
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz
Zzzz


In [28]:
pickle.dump(expansions, open( "expansions.p", "wb+" ) )
pickle.dump(check, open("check.p","wb+"))
pickle.dump(notgames, open("notgames.p","wb+"))

In [27]:
req.content

b'<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="1920">\n         <thumbnail>https://cf.geekdo-images.com/thumb/img/iXdPcg9THkD6EjL47RCTsFiHgDo=/fit-in/200x150/pic699535.jpg</thumbnail>\n      <image>https://cf.geekdo-images.com/original/img/6X9NW67kfXeL220H0vh-meXIbcE=/0x0/pic699535.jpg</image>\n                                     \t\t\t\t\n\t\t\t\t<name type="primary" sortindex="5" value="The Business Game" />\n\t\t\t\n\t\t\t\t\t\t                               \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Business Game" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Jeu d&#039;Affaires" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Mine A Million" />\n\t\t\t\n\t\t\t\t\t\t               \t\t\t\t\t\t\t\t\t\t\t\t\t<description>Board of two joined leaves, 12 mine derricks (2 of each of 6 colours), 6 lorries [trucks], 5 barges, 2 sh

In [22]:
req.content

b'<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="1920">\n         <thumbnail>https://cf.geekdo-images.com/thumb/img/iXdPcg9THkD6EjL47RCTsFiHgDo=/fit-in/200x150/pic699535.jpg</thumbnail>\n      <image>https://cf.geekdo-images.com/original/img/6X9NW67kfXeL220H0vh-meXIbcE=/0x0/pic699535.jpg</image>\n                                     \t\t\t\t\n\t\t\t\t<name type="primary" sortindex="5" value="The Business Game" />\n\t\t\t\n\t\t\t\t\t\t                               \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Business Game" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Jeu d&#039;Affaires" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Mine A Million" />\n\t\t\t\n\t\t\t\t\t\t               \t\t\t\t\t\t\t\t\t\t\t\t\t<description>Board of two joined leaves, 12 mine derricks (2 of each of 6 colours), 6 lorries [trucks], 5 barges, 2 sh

address = f'https://boardgamegeek.com/xmlapi2/thing?id=33&type=boardgame&stats=1'
req = requests.get(address)
req.content
# instantiate_games(req)

In [28]:
req.content

b'<error>\n    <message>Rate limit exceeded.</message>\n</error>'

In [15]:
pickle.dump(expansions, open( "expansions.p", "wb+" ) )

In [18]:
pickle.dump(check, open("check.p","wb+") )

In [20]:
print(inspector.get_columns('games'))

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'description', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'ratingscount', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'avgrating', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'published', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'maxplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'best', 'type': INTEGER(), 'nullable

In [36]:
current_ids = [game.id for game in session.query(Game).all()]

In [44]:
current_ids[:5]

[1, 2, 3, 4, 5]

In [43]:
len(current_ids)

1561

In [10]:
from sqlalchemy import inspect

inspector = inspect(engine)

In [11]:
print(inspector.get_table_names())

['artists', 'categories', 'games', 'games_artists', 'games_categories', 'games_mechanics', 'mechanics']


In [12]:
print(inspector.get_columns('games'))

[{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}, {'name': 'name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'description', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'ratingscount', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'avgrating', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'published', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'minplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'maxplayers', 'type': INTEGER(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}, {'name': 'best', 'type': INTEGER(), 'nullable

In [58]:
df = pd.read_sql_query("SELECT * FROM games", engine)
df.tail()

,id,name,description,published,minplayers,maxplayers,suggested_numplayers,playingtime,minplaytime,maxplaytime,minage,suggested_playerage,language_dependence,designer
80,391,Ocean,The idea is for players to populate the variou...,1999,2,6,None,20,20,20,8,None,None,None
81,431,Alhambra: The Card Game,Alhambra: The Card Game is a game that both in...,1992,3,6,None,45,45,45,10,None,None,None
82,451,Beutelschneider,"Translated as Cutpurse, this is a fairly uniqu...",1997,3,4,None,30,30,30,0,None,None,None
83,470,Road to the White House,extrapolated from the preview:&#10;&#10;The es...,1992,3,6,None,240,240,240,12,None,None,None
84,490,Warangel,"Warangel is a light, fantasy wargame, comprisi...",2000,2,6,None,60,60,60,12,None,None,None


In [99]:
df

,id,name,description,published,minplayers,maxplayers,suggested_numplayers,playingtime,minplaytime,maxplaytime,minage,suggested_playerage,language_dependence,designer
0,1,Die Macher,Die Macher is a game about seven sequential po...,1986,3,5,None,240,240,240,14,None,None,None
1,2,Dragonmaster,Dragonmaster is a trick-taking card game based...,1981,3,4,None,30,30,30,12,None,None,None
2,3,Samurai,"Part of the Knizia tile-laying trilogy, Samura...",1998,2,4,None,60,30,60,10,None,None,None
3,4,Tal der Könige,When you see the triangular box and the luxuri...,1992,2,4,None,60,60,60,12,None,None,None
4,5,Acquire,"In Acquire, each player strategically invests ...",1964,2,6,None,90,90,90,12,None,None,None
5,6,Mare Mediterraneum,"In the ancient lands along the Mediterranean, ...",1989,2,6,None,240,240,240,12,None,None,None
6,7,Cathedral,"In Cathedral, each player has a set of pieces ...",1978,2,2,None,20,20,20,8,None,None,None
7,8,Lords of Creation,"In this interesting offering from Warfrog, pla...",1993,2,5,None,120,120,120,12,None,None,None
8,9,El Caballero,"Although referred to as a sequel to El Grande,...",1998,2,4,None,90,90,90,13,None,None,None
9,10,Elfenland,Elfenland is a redesign of the original White ...,1998,2,6,None,60,60,60,10,None,None,None


In [60]:
import matplotlib.pyplot as plt

In [95]:
df.query('name == "Beutelschneider"')['description'].iloc[0]

'Translated as Cutpurse, this is a fairly unique entry in the trick-taking genre.  Three of the suits are relatively normal, however, the cards that are worth points, in the form of gold coins, have the least value.  The special thing about this game is the trump suit, which could have the Assassin killing the King or the Adventurer seducing the Countess; the character interaction is very interesting aspect.&#10;&#10;'

In [94]:
print(df.query("maxplaytime == 0")['playingtime'].iloc[0])

0


In [104]:
int('5+')

ValueError: invalid literal for int() with base 10: '5+'